# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм вида: `<дата> - <заголовок> - <ссылка>`

In [189]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [116]:
KEYWORDS = ['python', 'парсинг']
habr_url = 'https://habr.com/ru/all/'

In [123]:
def parse_link(url):
    req = requests.get(url)
    soup = BeautifulSoup(req.text)
    posts = soup.find_all('article', class_='post post_preview')
    preview_df = pd.DataFrame()

    for post in posts:
        read_more_a = post.find('a', class_='post__habracut-btn')
        read_more_url = read_more_a.get('href')
        read_more_req = requests.get(read_more_url)
        text_soup = BeautifulSoup(read_more_req.text)
        content = text_soup.find('div', id='post-content-body')

        if any([k in content.text.lower() for k in KEYWORDS]):
            post_date = post.find('span', class_='post__time').text # TODO: parse to datetime
            a = post.find('a', class_='post__title_link')
            title = a.text
            link = a.get('href')
            row = {'date': post_date, 'title': title, 'link': link, 'text': content.text}

            preview_df = pd.concat([preview_df, pd.DataFrame([row])])  

    return preview_df.reset_index()


In [113]:
parse_link(habr_url)

,index,date,title,link
0,0,сегодня в 18:35,"Как саботировать данные, которые технологическ...",https://habr.com/ru/company/madrobots/blog/548...


### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.  

Итоговый датафрейм формировать со столбцами: `<дата> - <заголовок> - <ссылка> - <текст_статьи>`


In [120]:
def get_all_links(URL, pages_count):
    if pages_count == 1:
        return [URL]

    result = []
    for i in range(1, pages_count):
        if i == 1:
            result.append(URL)
        else:
            result.append(f'{URL}page{i}')

    return result

In [121]:
all_links = get_all_links(habr_url, 4)
all_links

['https://habr.com/ru/all/',
 'https://habr.com/ru/all/page2',
 'https://habr.com/ru/all/page3']

In [122]:
def parse_habr(url, pages):
    all_links = get_all_links(url, pages)

    result_df = pd.DataFrame()
    for link in all_links:
        df = parse_link(link)
        result_df = pd.concat([result_df, df])

    return result_df

In [124]:
parse_habr(habr_url, 3)

,index,date,title,link,text
0,0,сегодня в 19:56,Три мои самые большие ошибки в карьере разрабо...,https://habr.com/ru/company/skillfactory/blog/...,Все мы знаем истории о программистах — рок-звё...
1,0,сегодня в 18:11,Уход от проблемы TTL или Стратегии корректного...,https://habr.com/ru/company/skillfactory/blog/...,Определение TTL для некоторых кэшированных дан...
2,0,сегодня в 17:19,Зачем мы транспилируем Haskell в JavaScript,https://habr.com/ru/company/typeable/blog/548574/,"Привет, Хабр! Сегодня мы расскажем, почему мы ..."


## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: `<почта> - <дата утечки> - <источник утечки> - <описание утечки>`  

**Подсказка**: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [206]:
import json
import time

In [245]:
EMAILS = ['best00@mail.ru', 'star55@gmail.com', 'wellknown@gmail.com']
AVAST_URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

In [230]:
mock_response = {
  "breaches": {
    "12": {
      "breachId": 12,
      "site": "vk.com",
      "recordsCount": 110121799,
      "description": "test",
      "publishDate": "2016-10-29T00:00:00Z",
      "statistics": {
        "usernames": 0,
        "passwords": 107425364,
        "emails": 104247327
      }
    }
  },
  "data": {
    "vk.com": {
      "best00@mail.ru": [
        {
          "breachId": 12,
          "usernameBreached": True,
          "passwordBreached": True
        }
      ]
    }
  },
  "summary": {
    "best00@mail.ru": {
      "breaches": [
        12
      ]
    }
  }
}

In [234]:
def build_row(response):
    email = list(response['summary'].keys())[0]
    breach = response['summary'][email]['breaches'][0]
    data = response['breaches'][str(breach)]
    row = {'email': email, 'date': data['publishDate'], 'breach source': data['site'], 'description': data['description']}
    return row


In [233]:
assert build_row(mock_response) == {'email': 'best00@mail.ru', 'date': '2016-10-29T00:00:00Z', 'breach source': 'vk.com', 'description':'test'}

best00@mail.ru
12


In [248]:
def prepare_headers():
    return {
  'Connection': 'keep-alive',
  'sec-ch-ua': '"Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99"',
  'Vaar-Header-App-Build-Version': '1.0.0',
  'sec-ch-ua-mobile': '?0',
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36',
  'Content-Type': 'application/json;charset=UTF-8',
  'Accept': 'application/json, text/plain, */*',
  'Vaar-Version': '0',
  'Vaar-Header-App-Product': 'hackcheck-web-avast',
  'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
  'Origin': 'https://www.avast.com',
  'Sec-Fetch-Site': 'same-site',
  'Sec-Fetch-Mode': 'cors',
  'Sec-Fetch-Dest': 'empty',
  'Referer': 'https://www.avast.com/',
  'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7,und;q=0.6'
}

In [252]:
def get_hack_check_df(url, emails):
    headers = prepare_headers()
    breaches_df = pd.DataFrame()

    for email in emails:
        time.sleep(2.0)
        payload = {'emailAddresses': [email]}
        res = requests.post(url, data=json.dumps(payload), headers=headers)
        print('status_code', res.status_code)
        print('reason', res.reason)
        print('text', res.text)
        response = res.json()
        
        if not response:
            print('Received empty response. May be you blocked for some time.')
            continue

        row = build_row(response)
        breaches_df = pd.concat([breaches_df, pd.DataFrame([row])])

    return breaches_df


In [263]:
get_hack_check_df(AVAST_URL, EMAILS)

status_code 200
reason OK
text {"breaches":{"12":{"breachId":12,"site":"vk.com","recordsCount":110121799,"description":"Popular Russian social networking platform VKontakte was breached in late 2012. Over 100 million clear-text passwords were compromised in the breach. Breached credential sets included victims' e-mail addresses, passwords, dates of birth, phone numbers and location details. The credential set was advertised on a dark web marketplace as of June 2016 for a price of one bitcoin. ","publishDate":"2016-10-29T00:00:00Z","statistics":{"usernames":0,"passwords":107425364,"emails":104247327}}},"data":{"vk.com":{"best00@mail.ru":[{"breachId":12,"usernameBreached":true,"passwordBreached":true}]}},"summary":{"best00@mail.ru":{"breaches":[12]}}}
status_code 200
reason OK
text {"breaches":{"16587":{"breachId":16587,"site":"dubsmash.com","recordsCount":161162574,"description":"In December 2018, Dubsmash's database was allegedly breached. The stolen data contains usernames, passwords,

,email,date,breach source,description
0,best00@mail.ru,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
0,star55@gmail.com,2019-03-07T00:00:00Z,dubsmash.com,"In December 2018, Dubsmash's database was alle..."
0,wellknown@gmail.com,2019-03-07T00:00:00Z,dubsmash.com,"In December 2018, Dubsmash's database was alle..."


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)  
```
GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ  
```

В итоге должен формироваться датафрейм со столбцами: `<дата поста> - <текст поста>`

In [260]:
GROUP = 'netology'  
TOKEN = 'MY_SECURE_TOKEN'
URL = "https://api.vk.com/method/wall.get"

In [273]:
params = {
    'access_token': TOKEN,
    'domain': GROUP,
    'v': '5.84'
    }
resp = requests.get(URL, params)

In [274]:
vk_df = pd.DataFrame(resp.json()['response']['items'])
vk_df.head()

,id,from_id,owner_id,date,marked_as_ads,post_type,text,is_pinned,attachments,comments,likes,reposts,views
0,80448,-30159897,-30159897,1615987503,0,post,Весна — лучшее время для перемен. Чтобы узнать...,1.0,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'count': 566},{'count': 23},{'count': 8},{'count': 4275}
1,81140,-30159897,-30159897,1616570820,0,post,🚀 Запустили бесплатный курс «Основы Figma» \n ...,NaN,"[{'type': 'link', 'link': {'url': 'https://net...",{'count': 0},{'count': 5},{'count': 3},{'count': 333}
2,81120,-30159897,-30159897,1616508960,0,post,Если вы знаете английский язык — у вас могут б...,NaN,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'count': 1},{'count': 7},{'count': 3},{'count': 1547}
3,81106,-30159897,-30159897,1616487360,0,post,В новом выпуске НетоNews обсудим изменение поз...,NaN,"[{'type': 'link', 'link': {'url': 'https://net...",{'count': 0},{'count': 6},{'count': 1},{'count': 1511}
4,81008,-30159897,-30159897,1616424360,0,post,Что общего между лентой новостей в Инстаграм и...,NaN,"[{'type': 'link', 'link': {'url': 'https://net...",{'count': 0},{'count': 8},{'count': 5},{'count': 2054}


In [275]:
vk_df[['date','text']].head()

,date,text
0,1615987503,Весна — лучшее время для перемен. Чтобы узнать...
1,1616570820,🚀 Запустили бесплатный курс «Основы Figma» \n ...
2,1616508960,Если вы знаете английский язык — у вас могут б...
3,1616487360,В новом выпуске НетоNews обсудим изменение поз...
4,1616424360,Что общего между лентой новостей в Инстаграм и...


#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам?
Прежде чем задать вопрос необходимо попробовать найти ответ самому в интернете. Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.

Любой вопрос должен быть сформулирован по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.
